## Interface utilisateur pour Sakila
1. Créer un foncrtion connect_db() qui prendra en entrée un dictionnaire contenant un nom d’utilisateur, un mot de passe, un host et le nom d’une base de données.
2. Créer une fonction new_customer() qui va permettre l’insertion d’un nouveau client dans la table customer en demandant à un utilisateur de saisir les données, ainsi que son adresse pour remplir les tables address, city et country. Faire attention au contrainte d’intégrité.

In [2]:
from sqlalchemy import create_engine
import pandas as pd 

def connect_db(user, password, host, bd):
    s = "mysql+pymysql://"
    s += user+":"+password+"@"+host+"/"+bd
    return create_engine(s)

def new_customer(engine):

    print("Informations client")
    nom = (input("Nom :")).upper()
    print(nom)
    prenom = (input("Prénom :")).upper()
    print(prenom)
    email = input("Email :")
    print(email)
    phone = input("Téléphone :")
    print(phone)
    print("Adresse")
    address = input("Adresse (ligne1) :")
    print(address)
    address2 = input("Adresse (ligne2) :")
    print(address2)
    city = input("Ville :")
    print(city)
    postal_code = input("Code postal :")
    print(postal_code)
    country = input("Pays : ")
    print(country)

    listCountries = pd.read_sql_query("SELECT * FROM country;", engine)

    if country not in list(listCountries["country"]):
        print("Pays pas présent")
        s = "INSERT INTO country (country) VALUES ('%s');" %(country)
        engine.execute(s)

    listCities = pd.read_sql_query("SELECT * FROM city;", engine)

    if city not in list(listCities["city"]):
        print("Ville pas présente")
        s = "INSERT INTO city (city, country_id) VALUES ('%s', (SELECT country_id FROM country WHERE country = '%s' LIMIT 1) );" %(city,country)
        engine.execute(s)


    listAddresses = pd.read_sql_query("SELECT * FROM address;", engine)
    listAddresses = listAdresses.fillna(" ")
    listAddresses["concatenee"] = listAddresses["address"]+" "+listAddresses["address2"]+" "+listAddresses["postal_code"]

    if (address+" "+address2+" "+postal_code) not in list(listAddresses["concatenee"]):
        s = "INSERT INTO address (address, address2, district, city_id, postal_code, phone, location) VALUES ('%s', '%s', ' ', (SELECT city_id FROM city WHERE city = '%s' LIMIT 1), '%s', '%s', ST_GeomFromWKB(X'0101000000000000000000F03F000000000000F03F'));" %(address, address2, city, postal_code, phone)
        engine.execute(s)


    s = "INSERT INTO customer (store_id, first_name, last_name, email, address_id, active, create_date) "
    s += "VALUES (1, '%s', '%s', '%s', (SELECT address_id FROM address WHERE address = '%s' AND address2 = '%s' AND postal_code = '%s'), 1, CURRENT_DATE());" %(nom, prenom, email, address, address2, postal_code)

    return engine.execute(s)

In [3]:
sakila = connect_db("julien", "simplon21", "localhost", "sakila")

#queryNewCustomer = new_customer(sakila)

3. Ecrire une fonction qui va afficher l’historique de location d’un client.



In [11]:
import pandas as pd

def historique(nom,prenom,engine):

    s = "SELECT r.rental_date, f.title "
    s += "FROM customer as c "
    s += "JOIN rental as r ON c.customer_id = r.customer_id "
    s += "JOIN inventory as i ON r.inventory_id = i.inventory_id "
    s += "JOIN film as f ON i.film_id = f.film_id "
    s += "WHERE (concat(c.first_name,' ', c.last_name) = '%s %s')" %(prenom.upper(),nom.upper())

    histo = pd.read_sql_query(s, engine)

    return histo

In [12]:
sakila = connect_db("julien", "simplon21", "localhost", "sakila")

hist = historique("smith","mary",sakila)
hist

,rental_date,title
0,2005-08-01 08:51:04,ADAPTATION HOLES
1,2005-07-29 03:58:49,AMISTAD MIDSUMMER
2,2005-06-18 08:41:48,ATTACKS HATE
3,2005-08-22 20:03:46,BIKINI BORROWERS
4,2005-06-16 15:18:57,CLOSER BANG
5,2005-07-09 16:38:01,CONFIDENTIAL INTERVIEW
6,2005-08-18 03:57:29,DALMATIONS SWEDEN
7,2005-06-15 18:02:53,DETECTIVE VISION
8,2005-07-28 09:04:45,DOORS PRESIDENT
9,2005-07-11 10:13:46,EXPECATIONS NATURAL


4. D’après vous quels seraient les indicateurs graphiques les plus pertinents pour suivre
l’évolution du business de location de vidéos ?
- Avec Matplotlib et Seaborn rédiger trois fonctions qui vont vous permettre
d’afficher les trois graphiques permettant de suivre au mieux la rentabilité de
l’affaire.